Однонаправленная LSTM

Импортируем необходимые зависимости

In [1]:
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

Загрузим текст для обучения из txt файла

In [37]:
with open('451.txt', 'r', encoding='windows-1251') as file:
    text = file.read()

Создадим токенизатор и словарь слов

In [15]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\asus
[nltk_data]     rog\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [38]:
tokens = word_tokenize(text.lower())
vocab = {word: i+1 for i, word in enumerate(set(tokens))}
encoded_text = [vocab[word] for word in tokens]

# Определим собственный Dataset для данных
class TextDataset(Dataset):
    def __init__(self, text, seq_length):
        self.text = text
        self.seq_length = seq_length
    
    def __len__(self):
        return len(self.text) - self.seq_length
    
    def __getitem__(self, idx):
        return (torch.tensor(self.text[idx:idx+self.seq_length], dtype=torch.long),
                torch.tensor(self.text[idx+self.seq_length], dtype=torch.long))



Создадим обучающие примеры

Создадим и обучим модель

In [39]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])
        return out

In [40]:
vocab_size = len(vocab) + 1  # размер словаря
embedding_dim = 128
hidden_dim = 128
output_dim = vocab_size
seq_length = 50
batch_size = 32

# Создаем Dataset и DataLoader
dataset = TextDataset(encoded_text, seq_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Инициализируем модель, функцию потерь и оптимизатор
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Процесс обучения
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}')



Epoch 1, Loss: 6.95306219498804
Epoch 2, Loss: 6.090884950750932
Epoch 3, Loss: 5.51571098475856
Epoch 4, Loss: 4.862567483281797
Epoch 5, Loss: 4.203318611488264
Epoch 6, Loss: 3.5787203594707027
Epoch 7, Loss: 3.0066154988021694
Epoch 8, Loss: 2.495000257082512
Epoch 9, Loss: 2.0501089744528866
Epoch 10, Loss: 1.6944604362446838


Попробуем сгенерировать текст

In [41]:
import torch.nn.functional as F

def generate_text(model, start_text, vocab, inv_vocab, seq_length=50, gen_length=100):
    model.eval()  # переводим модель в режим оценки
    
    # Токенизация стартового текста
    tokens = word_tokenize(start_text.lower())
    input_seq = [vocab.get(word, 0) for word in tokens]
    
    # Генерация текста
    generated_text = start_text
    
    for _ in range(gen_length):
        input_tensor = torch.tensor(input_seq[-seq_length:]).unsqueeze(0)  # добавляем batch dimension
        with torch.no_grad():
            output = model(input_tensor)
        
        # Предсказание следующего слова
        probs = F.softmax(output, dim=-1)
        next_word_idx = torch.argmax(probs, dim=-1).item()
        next_word = inv_vocab.get(next_word_idx, '<unk>')
        
        # Добавляем новое слово в последовательность
        generated_text += ' ' + next_word
        input_seq.append(next_word_idx)
    
    return generated_text



In [46]:
# Создание обратного словаря (индекс -> слово)
inv_vocab = {i: word for word, i in vocab.items()}

# Генерация текста
start_text = "Начнем"
generated_text = generate_text(model, start_text, vocab, inv_vocab, seq_length=50, gen_length=100)
print(generated_text)

Начнем что ночью он вздрогнул « обществом » , а может быть , кожей лица и тыльной стороны ладоней он именно в этом месте ощущал некое потепление воздуха , ибо невидимка одним своим присутствием мог на пять-шесть градусов поднять температуру окружающей его атмосферы , на мир , как пустота , на определенные аминокислоты . – она погремела в пригоршне каштанами , чтобы коснуться щеки . монтаг , дрожа , как на мир , а глаза , а целых трех ? – и перескок он . – но многие же боятся . они просто ушли , чтобы искрой зажигателя воспламенить керосин .
